# A new IMG

![](https://images.seeklogo.com/logo-png/14/1/umbrella-corporation-residentevil-logo-png_seeklogo-144673.png)

In [ ]:
%pip install -q ipywidgets

In [ ]:
# FeatureSlider.ipynb — robust: builds 3 usable features even if CSV has fewer
from __future__ import annotations
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as W

# -----------------------------
# 1) Data loading
# -----------------------------
def load_meta() -> tuple[pd.DataFrame, str]:
    candidates = [
        "data/meta.csv",            # typical in JupyterLite (content/ root)
        "content/data/meta.csv",    # if run from repo root
        "/files/data/meta.csv",
        "/drive/data/meta.csv",
    ]
    last_err = None
    for p in candidates:
        try:
            return pd.read_csv(p), p
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Could not load meta.csv; tried {candidates}\nLast error: {last_err}")

# -----------------------------
# 2) Numeric detection & feature synthesis
# -----------------------------
def _unit_col(df: pd.DataFrame) -> str | None:
    return next((c for c in ["unit","unit_id","Unit","id","neuron_id"] if c in df.columns), None)

def _coerce_numeric(series: pd.Series) -> pd.Series:
    # Make numeric-looking object columns numeric
    return pd.to_numeric(series, errors="coerce")

def detect_or_build_features(df_in: pd.DataFrame) -> tuple[pd.DataFrame, str | None, list[str], list[str]]:
    """
    Returns (df, unit_col, feature_cols[3], notes)
    - Converts numeric-looking columns
    - If <3 numeric columns, synthesizes extras to reach exactly 3
    """
    df = df_in.copy()
    ucol = _unit_col(df)

    # Prefer conventional names if all present
    preferred_sets = [
        ["feature1","feature2","feature3"],
        ["Feature1","Feature2","Feature3"],
        ["feature_1","feature_2","feature_3"],
        ["feat1","feat2","feat3"],
        ["F1","F2","F3"],
    ]
    for cols in preferred_sets:
        if all(c in df.columns for c in cols):
            for c in cols:
                df[c] = _coerce_numeric(df[c])
            notes = [f"Using preferred columns: {', '.join(cols)}"]
            return df, ucol, cols, notes

    # Otherwise: find numeric(-ish) columns (allow object that coerces to numeric)
    numeric_cols = []
    for c in df.columns:
        if c == ucol:
            continue
        s_num = _coerce_numeric(df[c])
        if s_num.notna().mean() >= 0.95:     # ≥95% numeric after coercion
            df[c] = s_num
            numeric_cols.append(c)

    notes = []
    # If we already have >=3, take the first 3
    if len(numeric_cols) >= 3:
        feat_cols = numeric_cols[:3]
        notes.append(f"Using first three numeric columns: {', '.join(feat_cols)}")
        return df, ucol, feat_cols, notes

    # If we have 2, synthesize a third as their mean
    if len(numeric_cols) == 2:
        a, b = numeric_cols
        f3 = f"feature3_mean({a},{b})"
        df[f3] = (df[a] + df[b]) / 2.0
        feat_cols = [a, b, f3]
        notes.append(f"Detected two numeric columns: {a}, {b}. Added {f3} = mean({a},{b}).")
        return df, ucol, feat_cols, notes

    # If we have 1, synthesize two: z-score and min-max
    if len(numeric_cols) == 1:
        a = numeric_cols[0]
        s = df[a]
        std = float(s.std(ddof=0)) or 1.0
        rng = float(s.max() - s.min()) or 1.0
        f2 = f"feature2_zscore({a})"
        f3 = f"feature3_minmax({a})"
        df[f2] = (s - float(s.mean())) / std
        df[f3] = (s - float(s.min())) / rng
        feat_cols = [a, f2, f3]
        notes.append(f"Detected one numeric column: {a}. Added {f2} and {f3}.")
        return df, ucol, feat_cols, notes

    # If none, fabricate from the index (last-resort)
    n = len(df)
    df["feature1_index"] = np.arange(n)
    df["feature2_index^0.5"] = np.sqrt(np.arange(n))
    df["feature3_sin(index)"] = np.sin(np.arange(n))
    feat_cols = ["feature1_index", "feature2_index^0.5", "feature3_sin(index)"]
    notes.append("No numeric columns detected; generated index-based features.")
    return df, ucol, feat_cols, notes

# -----------------------------
# 3) Plot -> PNG bytes (no display())
# -----------------------------
def plot_feature_png(df: pd.DataFrame, feature_cols: list[str], feature_index: int, unit_col: str | None) -> bytes:
    import io
    idx = int(feature_index) - 1
    if not (0 <= idx < len(feature_cols)):
        raise IndexError(f"feature_index {feature_index} out of range (1..{len(feature_cols)})")
    col = feature_cols[idx]
    y = df[col].to_numpy()
    x = np.arange(len(y))

    fig, ax = plt.subplots(figsize=(7.2, 4.5))
    ax.plot(x, y, marker="o", linestyle="-")
    ax.set_title(f"{col} across all units (n={len(y)})")
    ax.set_xlabel(unit_col if unit_col else "Unit index")
    ax.set_ylabel(col)
    ax.grid(True, alpha=0.3)
    fig.tight_layout()

    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=150, bbox_inches="tight")
    plt.close(fig)
    return buf.getvalue()

# -----------------------------
# 4) Build the small UI
# -----------------------------
def build_app():
    df_raw, resolved = load_meta()
    df, unit_col, feat_cols, notes = detect_or_build_features(df_raw)

    info = W.HTML(
        "<b>Loaded:</b> "
        f"<code>{resolved}</code><br>"
        f"<b>Using features:</b> {', '.join(feat_cols)}<br>"
        f"<small>{' '.join(notes)}</small>"
    )
    slider = W.IntSlider(value=1, min=1, max=3, step=1, description="Feature", continuous_update=False)
    img = W.Image(format="png")  # will hold our rendered plot bytes

    def redraw(change=None):
        img.value = plot_feature_png(df, feat_cols, slider.value, unit_col)

    slider.observe(lambda ch: redraw(), names="value")
    redraw()  # initial render

    return W.VBox([info, slider, img], layout=W.Layout(width="900px"))

app = build_app()
app  # let the notebook render the widget (no display())
